<a href="https://colab.research.google.com/github/jduell12/DS-Unit-2-Applied-Modeling/blob/master/C_Wrangle_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 
- Explore tabular data for supervised machine learning
- Join relational data for supervised machine learning

# Explore tabular data for superviesd machine learning 🍌

Wrangling your dataset is often the most challenging and time-consuming part of the modeling process.

In today's lesson, we’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!

Let’s get set up:

In [1]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://lambdaschool-ds-instruction.s3.amazonaws.com/datasets%3Ainstacart_2017_05_01.tar.gz')

Downloaded datasets%3Ainstacart_2017_05_01.tar.gz


In [2]:
# Uncompress data
import tarfile
tarfile.open('datasets%3Ainstacart_2017_05_01.tar.gz').extractall()

In [3]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [4]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename)

order_products__prior.csv
order_products__train.csv
aisles.csv
departments.csv
orders.csv
products.csv


In [5]:
import pandas as pd
from IPython.display import display

In [6]:
# For each csv file, look at its shape & head
def preview():
  for filename in glob('*.csv'):
    df = pd.read_csv(filename)
    print(filename, df.shape)
    display(df.head(5))


preview()

order_products__prior.csv (32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


order_products__train.csv (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


aisles.csv (134, 2)


...

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


orders.csv (3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


products.csv (49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


### The original task was complex ...

[The Kaggle competition said,](https://www.kaggle.com/c/instacart-market-basket-analysis/data):

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order.

> orders.csv: This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders.

Each row in the submission is an order_id from the test set, followed by product_id(s) predicted to be reordered.

> sample_submission.csv: 
```
order_id,products
17,39276 29259
34,39276 29259
137,39276 29259
182,39276 29259
257,39276 29259
```

### ... but we can simplify!

Simplify the question, from "Which products will be reordered?" (Multi-class, [multi-label](https://en.wikipedia.org/wiki/Multi-label_classification) classification) to **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

### Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Which customers have ordered this product before?
- How can we get a subset of data, just for these customers?
- What features can we engineer? We want to predict, will these customers reorder this product on their next order?

## Follow Along

### What was the most frequently ordered product?

In [7]:
order_products__train = pd.read_csv('order_products__train.csv')

In [8]:
order_products__train['product_id'].value_counts()

24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
44256        1
2764         1
4815         1
43736        1
46835        1
Name: product_id, Length: 39123, dtype: int64

In [9]:
temp = order_products__train.sort_values(by="product_id", ascending=False).head()

In [10]:
temp.groupby('product_id').count()

,order_id,add_to_cart_order,reordered
product_id,,,
49687,1,1,1
49688,4,4,4


In [11]:
order_products__train.groupby('product_id').order_id.count().sort_values(ascending=False)

product_id
24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
29560        1
8001         1
29566        1
8005         1
24836        1
Name: order_id, Length: 39123, dtype: int64

In [12]:
products = pd.read_csv('products.csv')
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [13]:
products[products.product_id==24852]

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


# Value Counts of all the product orders but get the names rather than the order Id

In [50]:
train = pd.merge(order_products__train, products, on='product_id')
train.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1,49302,1,1,Bulgarian Yogurt,120,16
1,816049,49302,7,1,Bulgarian Yogurt,120,16
2,1242203,49302,1,1,Bulgarian Yogurt,120,16
3,1383349,49302,11,1,Bulgarian Yogurt,120,16
4,1787378,49302,8,0,Bulgarian Yogurt,120,16


In [25]:
train['product_name'].value_counts()


Banana                                         18726
Bag of Organic Bananas                         15480
Organic Strawberries                           10894
Organic Baby Spinach                            9784
Large Lemon                                     8135
                                               ...  
Authentic Deep Dish, Spinach & Garlic Pizza        1
Green Cake Icing                                   1
Bread Crumbs, Plain                                1
Strawberry Cream Cheese                            1
Body Bar Soap Garden Mint                          1
Name: product_name, Length: 39123, dtype: int64

### How often are bananas included in a customer's next order?

There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

Customers' next orders are in the "train" and "test" sets. (The "prior" set has the orders prior to the most recent orders.)

We can't use the "test" set here, because we don't have its labels (only Kaggle & Instacart have them), so we don't know what products were bought in the "test" set orders.

So, we'll use the "train" set. It currently has one row per product_id and multiple rows per order_id.

But we don't want that. Instead we want one row per order_id, with a binary column: "Did the order include bananas?"

Let's wrangle!

In [26]:
train[train['order_id']==816049]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
1,816049,49302,7,1,Bulgarian Yogurt,120,16
26911,816049,22035,14,0,Organic Whole String Cheese,21,16
47740,816049,20574,2,1,Roasted Turkey,96,20
196442,816049,41220,4,0,Organic Romaine Lettuce,83,4
264132,816049,34969,1,1,Red Vine Tomato,83,4
355646,816049,7503,15,0,Whole Almonds,117,19
363747,816049,19446,16,0,Thick & Chunky Salsa,67,20
370530,816049,44142,8,0,Red Onion,83,4
552816,816049,36772,12,0,Bunny Pasta with Yummy Cheese Macaroni & Cheese,4,9
603546,816049,26604,11,0,Organic Blackberries,24,4


In [51]:
train['bananas'] = train['product_name'] == 'Banana'

In [43]:
train['bananas'].value_counts(normalize=True)

False    0.986476
True     0.013524
Name: bananas, dtype: float64

In [54]:
train_wrangled = train.groupby('order_id')['bananas'].any().reset_index()

In [55]:
train_wrangled['bananas'].value_counts(normalize=True)

# How often are bananas included in a customer's next order?
# 14% chance of bananas included in customer's next order

False    0.857281
True     0.142719
Name: bananas, dtype: float64

# Join relational data for supervised machine learning

## Overview
Often, you’ll need to join data from multiple relational tables before you’re ready to fit your models.

### Which customers have ordered this product before?

- Customers are identified by `user_id`
- Products are identified by `product_id`

Do we have a table with both these id's? (If not, how can we combine this information?)

No we don't have a table with both these ids. Can join the tables to have both the user and product ids. 

In [56]:
train[train.bananas == True]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,bananas
129688,226,24852,2,0,Banana,24,4,True
129689,473,24852,2,0,Banana,24,4,True
129690,878,24852,2,1,Banana,24,4,True
129691,1042,24852,1,1,Banana,24,4,True
129692,1139,24852,1,1,Banana,24,4,True
...,...,...,...,...,...,...,...,...
148409,3419531,24852,2,1,Banana,24,4,True
148410,3419542,24852,6,0,Banana,24,4,True
148411,3419629,24852,5,1,Banana,24,4,True
148412,3420088,24852,9,1,Banana,24,4,True


In [57]:
bananas_order_ids = train[train.bananas == True].order_id

In [59]:
orders = pd.read_csv('orders.csv')

In [60]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [61]:
orders[orders.order_id.isin(bananas_order_ids)]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
25,1492625,2,train,15,1,11,30.0
485,698604,34,train,6,4,13,30.0
623,1864787,43,train,12,5,11,26.0
652,1947848,46,train,20,4,20,11.0
881,13318,56,train,13,1,9,6.0
...,...,...,...,...,...,...,...
3420581,681092,206181,train,15,1,13,30.0
3420692,2530125,206191,train,6,0,9,30.0
3420786,3383615,206196,train,5,0,15,30.0
3420933,1716008,206205,train,4,1,16,10.0


In [63]:
bananas_orders = orders[orders.order_id.isin(bananas_order_ids)]
bananas_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
25,1492625,2,train,15,1,11,30.0
485,698604,34,train,6,4,13,30.0
623,1864787,43,train,12,5,11,26.0
652,1947848,46,train,20,4,20,11.0
881,13318,56,train,13,1,9,6.0


In [66]:
bananas_user_ids = bananas_orders.user_id.unique()

In [68]:
# array of customers who have ordered bananas before 
bananas_user_ids

array([     2,     34,     43, ..., 206196, 206205, 206209])

## Follow Along

### How can we get a subset of data, just for these customers?

We want *all* the orders from customers who have *ever* bought bananas.

(And *none* of the orders from customers who have *never* bought bananas.)

In [73]:
orders.shape

(3421083, 7)

In [72]:
order_subset = orders[orders.user_id.isin(bananas_user_ids)]
order_subset.shape

(332661, 7)

### What features can we engineer? We want to predict, will these customers reorder bananas on their next order?

## Challenge

**Continue to clean and explore your data.** Can you **engineer features** to help predict your target? For the evaluation metric you chose, what score would you get just by guessing? Can you **make a fast, first model** that beats guessing?

We recommend that you use your portfolio project dataset for all assignments this sprint. But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset today. Follow the instructions in the assignment notebook. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!